In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pretty_midi
import visual_midi
import joblib
from os.path import getsize
from MIDIComposingAI.utils import piano_roll_to_pretty_midi
from MIDIComposingAI.create_dataset import *
from MIDIComposingAI.get_back_data import *
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier

### Get data

In [3]:
path = '../raw_data/pretty_midi'
directory = [file_name for file_name in listdir(path) if getsize(f'{path}/{file_name}') < 300_000]

for i, file_name in enumerate(directory):
    file = joblib.load(f'{path}/{file_name}')
    if i == 0:
        X, y = create_nparray_dataset(file, ratio=0.1, store=False)
    else:
        try:
            loaded = create_nparray_dataset(file, ratio=0.1, store=False)
            X = np.concatenate((X, loaded[0]))
            y = np.concatenate((y, loaded[1]))
            del([loaded, file])
        except:
            pass
    if i % 10 == 0:
        print(f'{X.shape[0]} observations')
    if X.shape[0] >= 300:
        break

14 observations
95 observations
240 observations


In [4]:
X = X.reshape((X.shape[0], -1))
y = y.reshape((y.shape[0], -1))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=2)

### Make predictions

#### Initiate, train and save our model

In [6]:
tree = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=2)
tree.fit(X_train, y_train)
joblib.dump(tree, '../Models/dt_classifier.joblib')
# tree = joblib.load('../Models/tree.joblib')

['../Models/dt_classifier.joblib']

#### Attempt to debug it

In [7]:
# example_file = []
# for file in directory:
#     if 200_000 > getsize(f'{path}/{file}'): # We don't want too big or too little files
#         example_file.append(joblib.load(f'{path}/{file}'))
#         break

In [8]:
# dir = 'first_pres_dataset'
# for i, file in enumerate(example_file):
#     create_nparray_dataset(file, dir, name=f'chose{i}')

In [9]:
# X_test, y_test = joblib.load(f'../raw_data/pandas_dataframes/first_pres_dataset/chose0')

In [10]:
# X_test, y_test = create_nparray_dataset(example_file[0], 'f', 'n', store=False)

In [11]:
# X_test = X_test.reshape((X_test.shape[0], -1))
# y_test = y_test.reshape((y_test.shape[0], -1))

#### True thing

In [12]:
# Making our predictions
predictions = tree.predict(X_test)

In [13]:
# Test for the "false predictions" problem
for i, pred in enumerate(predictions):
    if i % 10 == 0:
        print((pred == y_test[i]).mean())

0.392
0.494


In [14]:
predicted_melodies = np.array([assembled_target_to_melody(prediction) for prediction in predictions])
true_melodies = np.array([assembled_target_to_melody(melody) for melody in y_test])

In [15]:
# They should be different

In [16]:
X_test = X_test.reshape(-1, 128, 500)

In [17]:
predicted_melodies[8].sum()

6930.0

In [33]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(piano_roll_to_pretty_midi(X_test[4]))

Loading BokehJS ...

Column(id='3467', ...)

In [31]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(piano_roll_to_pretty_midi(predicted_melodies[4]))

Loading BokehJS ...

Column(id='2974', ...)

In [32]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(piano_roll_to_pretty_midi(true_melodies[4]))

Loading BokehJS ...

Column(id='3218', ...)

In [20]:
copy = X_test.copy()
copy_2 = X_test.copy()

In [21]:
assembled_predicted_music = np.array(
    [assemblate_accompaniment_melody(test, predicted_melody) for test, predicted_melody in zip(copy, predicted_melodies)],
)

assembled_true_music = np.array(
    [assemblate_accompaniment_melody(test, true_melody) for test, true_melody in zip(copy_2, true_melodies)],
)

In [22]:
for i, (pred, true) in enumerate(zip(assembled_predicted_music, assembled_true_music)):
    try:
        piano_roll_to_pretty_midi(pred, fs=50).write(f'../PredVSTrue/Classifier/{i}_pred.mid')
        piano_roll_to_pretty_midi(true, fs=50).write(f'../PredVSTrue/Classifier/{i}_true.mid')
    except:
        pass